In [64]:
import pandas as pd

In [65]:
dfRegionalIB=pd.read_excel('MACA/MACA 2017 Installed base database End 2016_regional_files_All_Latin America.xlsx',
                           sheetname='Regional IB-List of units')

In [66]:
dfRegionalIB = dfRegionalIB[["GGO Region 13",
                             "Regional_Top_Down_61",
                             "Country",
                             "PLANT Name",
                             "UNIT",
                             "COMPANY (Plant Owner)",
                             "Parent Company",
                             "MW",
                             "Status at the end of 2016",
                             "Commisioning Year",
                             "Plant Type",
                             "Turbine Type",
                             "Turbine Short Name",
                             "Turbine Current Grouping",
                             "Generator Short Name",
                             "Generator Current Grouping",
                             "CITY",
                             "STATE",
                             "Age"]]

In [67]:
dfRegionalIB = dfRegionalIB[(dfRegionalIB['Plant Type'].isin(['Hydro PSP','Hydro Standard'])
                             & (dfRegionalIB['Status at the end of 2016']=='Commercial operation')
                             & (~dfRegionalIB['UNIT'].str.contains('MISSING HYDRO',case=False))
                            )]

In [68]:
dfRegionalIB[dfRegionalIB.columns[dfRegionalIB.isnull().any()].tolist()].isnull().sum()

PLANT Name                      40
COMPANY (Plant Owner)           30
Parent Company                3394
Commisioning Year              332
Turbine Type                   569
Turbine Short Name             917
Turbine Current Grouping       872
Generator Short Name          1839
Generator Current Grouping    1809
CITY                           662
STATE                          285
Age                            332
dtype: int64

In [69]:
import numpy as np
dfRegionalIB_num = dfRegionalIB.select_dtypes(['number'])
dfRegionalIB_num.apply(lambda x: (~(np.abs(x - x.mean()) <= 3*x.std()))).sum()

MW                   119
Commisioning Year    343
Age                  343
dtype: int64

In [70]:
def removeNumbers(series):
    series = series.str.replace('\d+', '')
    return series

In [71]:
def trimSeries(series):
    series = series.str.strip()
    return series

In [72]:
#Complete plant names based on unit names: 100% completed
dfRegionalIB['PLANT Name'] = dfRegionalIB['PLANT Name'].fillna(
    trimSeries(
        removeNumbers(
            dfRegionalIB['UNIT'])))

In [73]:
def fillNAMultiIndex(index_series,na_series):
    df = pd.concat([index_series,na_series],axis=1,join='inner')
    df = df.groupby(df.columns[0]).bfill()
    df = df.groupby(df.columns[0]).ffill()
    return df.iloc[:,1]

In [74]:
dfRegionalIB[dfRegionalIB.columns[dfRegionalIB.isnull().any()].tolist()].isnull().sum()

COMPANY (Plant Owner)           30
Parent Company                3394
Commisioning Year              332
Turbine Type                   569
Turbine Short Name             917
Turbine Current Grouping       872
Generator Short Name          1839
Generator Current Grouping    1809
CITY                           662
STATE                          285
Age                            332
dtype: int64

In [75]:
naColumnList = dfRegionalIB.columns[dfRegionalIB.isnull().any()].tolist()

for column in naColumnList:
    dfRegionalIB[column] = fillNAMultiIndex(dfRegionalIB['PLANT Name'],dfRegionalIB[column])    

In [76]:
dfRegionalIB[dfRegionalIB.columns[dfRegionalIB.isnull().any()].tolist()].isnull().sum()

COMPANY (Plant Owner)           23
Parent Company                3310
Commisioning Year              318
Turbine Type                   500
Turbine Short Name             805
Turbine Current Grouping       763
Generator Short Name          1666
Generator Current Grouping    1638
CITY                           629
STATE                          247
Age                            318
dtype: int64

In [77]:
#!!TORNAR MÉTODO DE DETECÇÃO DE OUTLIERS MAIS ROBUSTO!! Eliminar valores coerentes

def detectOutliers(series,std_dev_lim=3):
    import numpy as np
    seriesOutliers = series.loc[~(np.abs(series-series.mean())<=(3*series.std()))].sort_values().unique().tolist()
    return seriesOutliers

In [78]:
detectOutliers(dfRegionalIB['MW'])
# Outliers are OK

[310.0,
 315.0,
 320.0,
 340.0,
 350.0,
 360.0,
 364.0,
 365.0,
 375.0,
 380.0,
 400.0,
 402.0,
 410.0,
 430.0,
 502.0,
 527.0,
 620.0,
 700.0,
 730.0,
 740.0,
 750.0,
 770.0]

In [86]:
detectOutliers(dfRegionalIB['Commisioning Year'])
#Outliers are 0 and NaN

[1893.0, 1895.0, 1898.0, 1904.0, 1905.0, 1906.0, 1908.0, nan]

In [92]:
detectOutliers(dfRegionalIB['Age'])
#Outliers are 2017 and NaN

[109.0, 111.0, 112.0, 113.0, 119.0, 122.0, 124.0, nan]

In [93]:
dfRegionalIB.loc[dfRegionalIB['Commisioning Year']==0,'Commisioning Year']=2017

In [94]:
dfRegionalIB.loc[dfRegionalIB['Age']==2017,'Age']=0

In [95]:
detectOutliers(dfRegionalIB['Commisioning Year'])
#Outliers are OK

[1893.0, 1895.0, 1898.0, 1904.0, 1905.0, 1906.0, 1908.0, nan]

In [96]:
detectOutliers(dfRegionalIB['Age'])
#Outliers are OK

[109.0, 111.0, 112.0, 113.0, 119.0, 122.0, 124.0, nan]

In [106]:
dfRegionalIB.loc[((dfRegionalIB['Age'].isnull() & (dfRegionalIB['MW']>=5)),
                  ['PLANT Name','MW','Age','Commisioning Year'])]

,PLANT Name,MW,Age,Commisioning Year
648,CABEZA DE BOI,15.50,NaN,NaN
649,CABEZA DE BOI,15.50,NaN,NaN
1180,DA FAZENDA,10.24,NaN,NaN
1181,DA FAZENDA,10.24,NaN,NaN
3271,RIO TIMBO,5.50,NaN,NaN
3457,SALTO VOLTAO,8.20,NaN,NaN


In [112]:
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Age']=2016
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Age']=2016
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Age']=1959
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Age']=2001
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Commisioning Year']=1
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Commisioning Year']=1
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Commisioning Year']=58
dfRegionalIB.loc[dfRegionalIB['PLANT Name']=='CABEZA DE BOI','Commisioning Year']=16